In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import monai
from PIL import Image
import torch
from monai.visualize import blend_images, matshow3d, plot_2d_or_3d_image
from tqdm.notebook import tqdm
from medpy.metric.binary import hd, dc


In [ ]:
# Cursor parking space

In [ ]:
patient_folders = ["data/testing/testing/" + x + "/" for x in os.listdir("data/testing/testing/")]
# patient_folders = ["data/training/" + x + "/" for x in os.listdir("data/training/")]
patient_files = [[x + y[:-7] for y in os.listdir(x) if "frame" in y and "gt" not in y] for x in patient_folders]
patient_files_flattened = [element for sublist in patient_files for element in sublist]


images = [{'img': x} for x in patient_files_flattened]

In [ ]:
header_dict = {}

In [ ]:
from evaluate import load_nii
class LoadNIFTI(monai.transforms.Transform):
    """
    This custom Monai transform loads the data from the rib segmentation dataset.
    Defining a custom transform is simple; just overwrite the __init__ function and __call__ function.
    """
    def __init__(self, keys=None):
        pass

    def __call__(self, sample):
        img_file = sample['img'] + ".nii.gz"
        # img_mask = sample['img'] + "_gt.nii.gz"
                
        image, img_affine, img_header = load_nii(img_file)
        image = np.moveaxis(image, (2), (0))
        
        header_dict[sample['img']]=img_header
        
        # mask, mask_affine, mask_header = load_nii(img_mask)
        # mask = np.moveaxis(mask, (2), (0))
        
        return {'img': image, 'name': sample, 'affine': img_affine}
    
       
class ScaleDims(monai.transforms.Transform):
    def __init__(self, keys=None):
        pass
    
    def __call__(self, sample):
        scaling = sample['scaling']
        
        return monai.transforms.Zoomd(keys=['img', 'mask'], mode=['area', 'nearest'], zoom=(scaling[3] / 10, scaling[1] / 1.5, scaling[2] / 1.5), keep_size=False)(sample)
    
        # img = monai.transforms.Zoomd(keys=['img', 'mask']
class ScaleDimsInverse(monai.transforms.Transform):
    def __init__(self, keys=None):
        pass
    
    def __call__(self, sample):
        scaling = sample['scaling']
        
        return monai.transforms.Zoomd(keys=['img', 'mask'], mode=['area', 'nearest'], zoom=(1 / (scaling[3] / 10), 1 / (scaling[1] / 1.5), 1 / (scaling[2] / 1.5)), keep_size=False)(sample)
    
    
class FindCenter(monai.transforms.Transform):
    def __init__(self, keys=None):
        self.model = monai.networks.nets.Unet(
                spatial_dims=3,
                in_channels=1,
                out_channels=3,
                channels = (8, 16, 32, 64),
                strides=(1, 1, 1),
                num_res_units=2,
            ).to(device)
        
        self.model.load_state_dict(torch.load("models/trainedUNet1655808833.8215299_24.pt"))
        self.model.eval()
        
    def __call__(self, sample):
        img = sample['img']
        with torch.no_grad():
            out = self.model(torch.unsqueeze(torch.Tensor(img).to(device), dim=0))
            combined = np.sum(out[0].detach().cpu().numpy(), axis=(0, 1))
            cx, cy = ndi.center_of_mass(combined)
            dim_height = out.shape[2]
            sample['heart_pos'] = int(cx), int(cy)
            return monai.transforms.SpatialCropd(keys=['img', 'mask'], roi_size=(dim_height, 128, 128), roi_center=(dim_height // 2, int(cx), int(cy)))(sample)

class FindCenterInverse(monai.transforms.Transform):
    def __init__(self, keys=None):
        pass
    
    def __call__(self, sample):
        cx, cy = sample['heart_pos']
        
        
    


In [ ]:
# Define transforms for loading the dataset

# add_channels_transform = monai.transforms.AddChanneld(keys=['img', 'mask'])
# flip_transform = monai.transforms.RandFlipd(keys=['img', 'mask'], prob=1, spatial_axis=1)
# rotate_transform = monai.transforms.RandRotated(keys=['img', 'mask'], range_x=np.pi/4, prob=1, mode=['bilinear', 'nearest'])

# compose_transform = monai.transforms.Compose(
#     [
#         LoadNIFTI(),
#         monai.transforms.AddChanneld(keys=['img', 'mask']),
#         monai.transforms.ScaleIntensityd(keys=['img', 'mask'], minv=0.0, maxv=1.0),
#         SplitMask(),
#         monai.transforms.Resized(keys=['img', 'mask'], spatial_size=(-1, 128, 128)),
#         monai.transforms.SpatialPadd(keys=['img', 'mask'], spatial_size=(16, -1, -1)),
#         monai.transforms.SpatialCropd(keys=['img', 'mask'], roi_size=(16, 128, 128), roi_center=(8, 64, 64)),
#         monai.transforms.ScaleIntensityd(keys=['mask'], minv=0.0, maxv=1.0)
#     ]
# )

compose_transform = monai.transforms.Compose(
    [
        LoadNIFTI(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityd(keys=['img'], minv=0.0, maxv=1.0),
    ]
)


In [ ]:
train_dict_list = [x for x in images]
dataset = monai.data.CacheDataset(train_dict_list, transform=compose_transform)


Loading dataset: 100%|██████████| 100/100 [00:00<00:00, 164.45it/s]


In [ ]:
data_loader = monai.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
device = "cpu"

In [ ]:
# model = monai.networks.nets.UNETR(in_channels=1, out_channels=3, img_size=(16,128,128), feature_size=32, norm_name='batch').to(device)
model = monai.networks.nets.Unet(
    spatial_dims=3,
    in_channels=1,
    out_channels=3,
    channels = (8, 16, 32, 64),
    strides=(1, 1, 1),
    num_res_units=2,
).to(device)

model.load_state_dict(torch.load('models/trainedUNet1655751992.34428.pt'))

<All keys matched successfully>

In [ ]:
# def flatten(mask):
#     out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
#     return out

# import evaluate

# for d in data_loader:
#     img = d['img']
#     label = d['mask']
#     pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
#     # print(pred.shape)
#     flatt_pred = flatten(pred[0])
#     fixed_label = 3*(label[0][0])
#     print(flatt_pred.shape)
#     print(fixed_label.shape)
    
#     print(evaluate.metrics(flatt_pred, fixed_label, [1,1,1]))
    
#     # evaluate.metrics(label, pred, [1, 1, 1])

    
# #     plt.imshow(d['img'][0][0][4], cmap='gray')
# #     plt.show()
# #     out = torch.clamp(model(d['img'].to(device)), min=0, max=1).detach().cpu().numpy()
# #     o = np.concatenate((out[0][0][4], out[0][1][4], out[0][2][4]), axis=1)
    
# #     # a = np.expand_dims(flatten(d['mask'][0, :, 4]), axis=0)
# #     b = np.expand_dims(flatten(out[0])[4],axis=0)
    
# #     # plt.imshow(a[0])
# #     # plt.show()
# #     plt.imshow(b[0])
# #     plt.show()

#     # print(d['img'].shape)
#     # print(evaluate.metrics(a, b, [1, 1, 1]))
#     # break


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'nibabel.nifti1.Nifti1Header'>

In [ ]:
def flatten(mask):
    out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
    return out

def get_name(filename):
    patient = filename.rpartition('data/testing/testing/')[2][:10]
    if filename[-2:] == '01':
        state = '_ED'
    else:
        state = '_ES'
    return patient+state

import evaluate

for d in data_loader:
    
    img = d['img']
    name = get_name(d['name']['img'][0])
    header = header_dict[d['name']['img'][0]]
    affine = d['affine'][0]
    # print(affine.shape)
    pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
    flatt_pred = flatten(pred[0])
    
    evaluate.save_nii("results/masks/"+name, flatt_pred, affine, header)
    
    # print(pred.shape)
    


torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import monai
from PIL import Image
import torch
from monai.visualize import blend_images, matshow3d, plot_2d_or_3d_image
from tqdm.notebook import tqdm
from medpy.metric.binary import hd, dc


In [ ]:
# Cursor parking space

In [ ]:
patient_folders = ["data/testing/testing/" + x + "/" for x in os.listdir("data/testing/testing/")]
# patient_folders = ["data/training/" + x + "/" for x in os.listdir("data/training/")]
patient_files = [[x + y[:-7] for y in os.listdir(x) if "frame" in y and "gt" not in y] for x in patient_folders]
patient_files_flattened = [element for sublist in patient_files for element in sublist]


images = [{'img': x} for x in patient_files_flattened]

In [ ]:
header_dict = {}

In [ ]:
from evaluate import load_nii
class LoadNIFTI(monai.transforms.Transform):
    """
    This custom Monai transform loads the data from the rib segmentation dataset.
    Defining a custom transform is simple; just overwrite the __init__ function and __call__ function.
    """
    def __init__(self, keys=None):
        pass

    def __call__(self, sample):
        img_file = sample['img'] + ".nii.gz"
        # img_mask = sample['img'] + "_gt.nii.gz"
                
        image, img_affine, img_header = load_nii(img_file)
        image = np.moveaxis(image, (2), (0))
        
        header_dict[sample['img']]=img_header
        
        # mask, mask_affine, mask_header = load_nii(img_mask)
        # mask = np.moveaxis(mask, (2), (0))
        
        return {'img': image, 'name': sample, 'affine': img_affine}
    

class StoreNIFTI(monai.transforms.Transform):
    def __init__(self, keys=None):
        pass
    
    def __call__(self, sample):
        print(sample)
    
    


In [ ]:
# Define transforms for loading the dataset

# add_channels_transform = monai.transforms.AddChanneld(keys=['img', 'mask'])
# flip_transform = monai.transforms.RandFlipd(keys=['img', 'mask'], prob=1, spatial_axis=1)
# rotate_transform = monai.transforms.RandRotated(keys=['img', 'mask'], range_x=np.pi/4, prob=1, mode=['bilinear', 'nearest'])

# compose_transform = monai.transforms.Compose(
#     [
#         LoadNIFTI(),
#         monai.transforms.AddChanneld(keys=['img', 'mask']),
#         monai.transforms.ScaleIntensityd(keys=['img', 'mask'], minv=0.0, maxv=1.0),
#         SplitMask(),
#         monai.transforms.Resized(keys=['img', 'mask'], spatial_size=(-1, 128, 128)),
#         monai.transforms.SpatialPadd(keys=['img', 'mask'], spatial_size=(16, -1, -1)),
#         monai.transforms.SpatialCropd(keys=['img', 'mask'], roi_size=(16, 128, 128), roi_center=(8, 64, 64)),
#         monai.transforms.ScaleIntensityd(keys=['mask'], minv=0.0, maxv=1.0)
#     ]
# )

compose_transform = monai.transforms.Compose(
    [
        LoadNIFTI(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityd(keys=['img'], minv=0.0, maxv=1.0),
    ]
)


In [ ]:
train_dict_list = [x for x in images]
dataset = monai.data.CacheDataset(train_dict_list, transform=compose_transform)


Loading dataset: 100%|██████████| 100/100 [00:00<00:00, 164.45it/s]


In [ ]:
data_loader = monai.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
device = "cpu"

In [ ]:
# model = monai.networks.nets.UNETR(in_channels=1, out_channels=3, img_size=(16,128,128), feature_size=32, norm_name='batch').to(device)
model = monai.networks.nets.Unet(
    spatial_dims=3,
    in_channels=1,
    out_channels=3,
    channels = (8, 16, 32, 64),
    strides=(1, 1, 1),
    num_res_units=2,
).to(device)

model.load_state_dict(torch.load('models/trainedUNet1655751992.34428.pt'))

<All keys matched successfully>

In [ ]:
def flatten(mask):
    out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
    return out

import evaluate

for d in data_loader:
    img = d['img']
    label = d['mask']
    pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
    # print(pred.shape)
    flatt_pred = flatten(pred[0])
    fixed_label = 3*(label[0][0])
    print(flatt_pred.shape)
    print(fixed_label.shape)
    
    print(evaluate.metrics(flatt_pred, fixed_label, [1,1,1]))
    
    # evaluate.metrics(label, pred, [1, 1, 1])

    
#     plt.imshow(d['img'][0][0][4], cmap='gray')
#     plt.show()
#     out = torch.clamp(model(d['img'].to(device)), min=0, max=1).detach().cpu().numpy()
#     o = np.concatenate((out[0][0][4], out[0][1][4], out[0][2][4]), axis=1)
    
#     # a = np.expand_dims(flatten(d['mask'][0, :, 4]), axis=0)
#     b = np.expand_dims(flatten(out[0])[4],axis=0)
    
#     # plt.imshow(a[0])
#     # plt.show()
#     plt.imshow(b[0])
#     plt.show()

    # print(d['img'].shape)
    # print(evaluate.metrics(a, b, [1, 1, 1]))
    # break


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'nibabel.nifti1.Nifti1Header'>

In [ ]:
def flatten(mask):
    out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
    return out

def get_name(filename):
    patient = filename.rpartition('data/testing/testing/')[2][:10]
    if filename[-2:] == '01':
        state = '_ED'
    else:
        state = '_ES'
    return patient+state

import evaluate

for d in data_loader:
    
    img = d['img']
    name = get_name(d['name']['img'][0])
    header = header_dict[d['name']['img'][0]]
    affine = d['affine'][0]
    # print(affine.shape)
    pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
    flatt_pred = flatten(pred[0])
    
    evaluate.save_nii("results/masks/"+name, flatt_pred, affine, header)
    
    # print(pred.shape)
    


torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([4, 4])
torch.Size([